In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle
import shap
import joblib

In [ ]:
# ========================================
# CONFIGURATION
# ========================================
USE_EXTERNAL_VALIDATION = True  # True: run external validation | False: skip validation
SAVE_MODEL = True  # True: save model to .pkl file | False: skip saving

In [ ]:
try:
    # For Colab: upload file
    from google.colab import files
    uploaded = files.upload()
    filename = next(iter(uploaded))
    df = pd.read_csv(filename)
except:
    # For local execution: use predefined path
    df = pd.read_csv('../data/training/Oliynyk_Js_optunaFS.csv')

print(f"Loaded {len(df)} samples with {len(df.columns)} columns")
display(df.head())

In [ ]:
if USE_EXTERNAL_VALIDATION:
        # Configuration: Set target property and feature type
    property_name = "Js"  # "Hc", "Js", or "rho"
    featurization_method = "Oliynyk"  # "Composition", "WenAlloys", or "Oliynyk"
    output_file = f"predictions_{property_name}_{featurization_method}.xlsx"
    column_name = f"{property_name}_predict"

    # Load experimental dataset
    try:
        # Colab: upload file
        uploaded = files.upload()
        X_predict = pd.read_csv('oliynyk_Js_for_predict.csv')
    except:
        # Local: load from directory
        X_predict = pd.read_csv('../data/external_validation/oliynyk_Js_for_predict.csv')

    print(f"Loaded {len(X_predict)} experimental samples with {len(X_predict.columns)} features")
    display(X_predict.head())

    # Initialize prediction results table (5 alloys × 2 models)
    results = pd.DataFrame({
        "Model": ["ETR"]*5 + ["XGB"]*5,
        "Alloy_number": list(range(1, 6)) + list(range(1, 6)),
        column_name: [np.nan]*10
    })

    print(f"\nPrediction table initialized for {property_name} ({featurization_method})")
    print(results, "\n")

In [ ]:
# Separate features and target variable
df_drop_column = df.iloc[:, :-1]  # All columns except the last one (features)
y = df.loc[:, 'target']            # Target column only

# Creating Training Dataset

In [ ]:
# Rename features to abbreviated forms for better visualization
df_renamed = df_drop_column.rename(columns={
    'avg_l_quantum_number': 'avg q. num. l',
    'avg_group': 'avg group',
    'dev_Mendeleev_Number': 'Δ M. num.',
    'avg_Zunger_radii_sum': 'avg rad. Zunger',
    'sum_specific_heat_(J/g_K)_': 'Σ specific heat',
    'avg_crystal_radius': 'avg cryst. rad.'
})

In [ ]:
# Prepare data for training
features = df_renamed
target = y

# Shuffle data to ensure random distribution
features, target = shuffle(features, target, random_state=42)

# Split into training and test sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
# Initialize an empty DataFrame to store all results
model_results = pd.DataFrame(columns=['Model', 'MAE', 'MSE', 'RMSE', 'R²'])

# Prediction Properties

In [ ]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
import xgboost as xgb

# Random Forest

In [ ]:
# Train Random Forest model with pre-optimized hyperparameters
best_model = RandomForestRegressor(
    n_estimators=270,
    max_depth=10,
    min_samples_split=2,
    min_samples_leaf=1,
    criterion='friedman_mse',
    random_state=42
)
best_model.fit(X_train, y_train)

In [ ]:
if SAVE_MODEL:
    # Save trained model
    joblib.dump(best_model, '1_oliynyk_SP_rfr.pkl')
    print("✅ Model saved as '1_oliynyk_SP_rfr.pkl'")

    try:
        files.download('1_oliynyk_SP_rfr.pkl')
    except:
        pass

In [ ]:
# Predict on test set
y_pred = best_model.predict(X_test)

# Calculate evaluation metrics
test_r2 = r2_score(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)

# Display metrics in formatted table
print("\nTest Set Evaluation Metrics:")
print("----------------------------------------")
print(f"| {'Metric':<10} | {'Value':>15} |")
print("|------------|----------------|")
print(f"| R²         | {test_r2:>15.4f} |")
print(f"| MAE        | {test_mae:>15.4f} |")
print(f"| MSE        | {test_mse:>15.4f} |")
print(f"| RMSE       | {test_rmse:>15.4f} |")
print("----------------------------------------")

In [ ]:
def evaluate_and_store(model, model_name, X_test, y_test, results_df):
    """Evaluate model and store metrics in results DataFrame."""

    # Predict and calculate metrics
    y_pred = model.predict(X_test)

    metrics = {
        'Model': model_name,
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R²': r2_score(y_test, y_pred)
    }

    # Add to results
    results_df = pd.concat([results_df, pd.DataFrame([metrics])], ignore_index=True)
    return results_df


# Store RFR evaluation results
model_results = evaluate_and_store(best_model, 'RFR', X_test, y_test, model_results)

# Extra Trees Regressor

In [ ]:
# Train Extra Trees model with pre-optimized hyperparameters
best_model = ExtraTreesRegressor(
    n_estimators=270,
    max_depth=7,
    min_samples_split=4,
    min_samples_leaf=2,
    criterion='poisson',
    random_state=42
)
best_model.fit(X_train, y_train)

In [ ]:
if SAVE_MODEL:

# Save model to .pkl file
    joblib.dump(best_model, '2_oliynyk_SP_etr.pkl')

    print("✅ Model saved as '2_oliynyk_SP_etr.pkl'")
    try:
        files.download('2_oliynyk_SP_etr.pkl')
    except:
        pass

In [ ]:
# Predict on test set
y_pred = best_model.predict(X_test)

# Calculate evaluation metrics
test_r2 = r2_score(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)

# Display metrics in formatted table
print("\nTest Set Evaluation Metrics:")
print("----------------------------------------")
print(f"| {'Metric':<10} | {'Value':>15} |")
print("|------------|----------------|")
print(f"| R²         | {test_r2:>15.4f} |")
print(f"| MAE        | {test_mae:>15.4f} |")
print(f"| MSE        | {test_mse:>15.4f} |")
print(f"| RMSE       | {test_rmse:>15.4f} |")
print("----------------------------------------")

In [ ]:
if USE_EXTERNAL_VALIDATION:
    # Make predictions on external validation set (ETR)
    y_pred_etr = best_model.predict(X_predict)

    # Save ETR predictions to results DataFrame
    results.loc[results["Model"] == "ETR", column_name] = y_pred_etr
    print(f"ETR predictions saved: {y_pred_etr}")
    print(results)
    print("\n")

In [ ]:
def evaluate_and_store(model, model_name, X_test, y_test, results_df):
    """Evaluate model and store metrics in results DataFrame."""

    # Predict on test set
    y_pred = model.predict(X_test)

    # Calculate regression metrics
    metrics = {
        'Model': model_name,
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R²': r2_score(y_test, y_pred)
    }

    # Append metrics to results DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([metrics])], ignore_index=True)
    return results_df


# Evaluate ETR model and store results
model_results = evaluate_and_store(best_model, 'ETR', X_test, y_test, model_results)

In [ ]:
# ====================================================================
# SHAP BEESWARM VISUALIZATION
# ====================================================================

# Calculate SHAP values for all features using TreeExplainer
explainer = shap.TreeExplainer(best_model, X_train)
shap_values = explainer(X_train)


# Set global font parameters
mpl.rcParams.update({
    'font.size': 23,
    'axes.labelsize': 23,
    'xtick.labelsize': 23,
    'ytick.labelsize': 23,
})


# Create figure and axis
fig, ax = plt.subplots(figsize=(8, 6))


# Generate beeswarm plot showing feature importance and value distribution
shap.plots.beeswarm(
    shap_values,
    max_display=5,                     # Show top 5 most important features
    group_remaining_features=False,    # Don't group remaining features
    show=False,                        # Don't display yet (for customization)
    s=150,                             # Marker size
    ax=ax,
    plot_size=None
)


# Customize axis labels and ticks
ax.set_xlabel("SHAP Values", fontsize=23)
ax.tick_params(axis='x', labelsize=23, pad=2)
ax.tick_params(axis='y', which='both', length=0, pad=1, labelsize=23)


# Set custom x-axis tick positions
desired_ticks = [-0.4, -0.2, 0, 0.2]
ax.set_xticks(desired_ticks)


# Shift y-axis labels to the right
dx_pt = -3
offset = mtransforms.ScaledTranslation(dx_pt/72.0, 0, fig.dpi_scale_trans)
for t in ax.get_yticklabels():
    t.set_ha('right')
    t.set_transform(t.get_transform() + offset)


# Position y-axis spine
ax.spines['left'].set_position(('axes', 0.0))
ax.yaxis.set_ticks_position('left')


# Customize colorbar (feature value scale)
cax = fig.axes[-1]
cax.tick_params(pad=2, labelsize=23)
cax.set_ylabel("Feature Value", fontsize=23, labelpad=4)
cax.yaxis.set_label_coords(2.5, 0.5)


# Apply tight layout and save
fig.tight_layout(pad=0.2)
fig.savefig('Oliynyk_SHAP_beeswarm_Js_ETR.png', dpi=600, bbox_inches='tight', transparent=False)


plt.show()


# Download plot in Colab environment
try:
    files.download('Oliynyk_SHAP_beeswarm_Js_ETR.png')
except:
    pass  # Skip download for local execution

In [ ]:
def shifted_coolwarm_cmap(p, n=256):
    """Shift coolwarm colormap so position p becomes neutral center."""
    base = plt.get_cmap("coolwarm")
    xs = np.linspace(0, 1, n)

    cols = []
    for x in xs:
        if x <= p:
            t = 0.5 * (x / p) if p > 0 else 0.0
        else:
            t = 0.5 + 0.5 * ((x - p) / (1 - p)) if p < 1 else 1.0
        cols.append(base(t))

    return mcolors.ListedColormap(cols, name="coolwarm_shifted")


def make_coolwarm_saturated_with_center(values, vcenter=0.0, nbins=6, steps=(1, 2, 2.5, 3, 5, 10)):
    """Create norm, cmap, and ticks for SHAP with saturated colors and zero at neutral."""
    x = np.asarray(values, dtype=float)
    x = x[np.isfinite(x)]

    # Fallback for empty data
    if x.size == 0:
        norm = mcolors.Normalize(vmin=-1, vmax=1, clip=True)
        cmap = plt.get_cmap("coolwarm")
        ticks = np.array([-1, 0, 1], dtype=float)
        return norm, cmap, ticks

    data_min, data_max = float(np.min(x)), float(np.max(x))

    # Normalize by actual data range for saturated colors
    norm = mcolors.Normalize(vmin=data_min, vmax=data_max, clip=True)

    # Generate ticks
    loc = mticker.MaxNLocator(nbins=nbins, steps=list(steps))
    ticks = loc.tick_values(data_min, data_max)
    ticks = ticks[(ticks >= norm.vmin) & (ticks <= norm.vmax)]

    # Shift colormap to place vcenter at neutral
    if data_min < vcenter < data_max:
        p = (vcenter - data_min) / (data_max - data_min)
        cmap = shifted_coolwarm_cmap(p, n=256)

        # Ensure vcenter is in ticks
        if not np.any(np.isclose(ticks, vcenter)):
            ticks = np.sort(np.append(ticks, vcenter))
    else:
        cmap = plt.get_cmap("coolwarm")

    return norm, cmap, ticks

In [ ]:
# ====================================================================
# GRID-SHAP VISUALIZATION
# ====================================================================

# Calculate SHAP values using TreeExplainer
explainer = shap.TreeExplainer(best_model, X_train)
shap_values = explainer(X_train)

# Identify top 5 most important features by mean absolute SHAP value
mean_abs_shap_values = np.abs(shap_values.values).mean(axis=0)
top5_indices = np.argsort(mean_abs_shap_values)[-5:][::-1]

# Create figure with 3x2 grid layout
plt.style.use('default')
fig = plt.figure(figsize=(8, 10))
gs = GridSpec(3, 2, figure=fig, wspace=0.65, hspace=0.5)

# Generate scatter plots for each top feature
for i, feature_index in enumerate(top5_indices):
    feature_name = X_train.columns[feature_index]
    row = i // 2
    col = i % 2

    ax = fig.add_subplot(gs[row, col])
    ax.set_facecolor('#f0f0f0')

    # Extract SHAP values for current feature
    shap_col = shap_values[:, feature_index].values

    # Get normalized colormap with zero at neutral center
    norm, cmap_local, ticks = make_coolwarm_saturated_with_center(
        shap_col,
        vcenter=0.0,
        nbins=6
    )

    # Create scatter plot: feature value vs target, colored by SHAP
    sc = ax.scatter(
        X_train[feature_name],
        y_train,
        c=shap_col,
        cmap=cmap_local,
        norm=norm,
        alpha=0.9,
        s=40,
        edgecolor='black',
        linewidth=0.25,
        marker='o'
    )

    # Add colorbar with scientific notation
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(sc, cax=cax)

    # Configure colorbar ticks
    cbar.set_ticks(ticks)
    cbar.formatter = mticker.ScalarFormatter(useMathText=False)
    cbar.formatter.set_powerlimits((-100, 100))
    cbar.update_ticks()
    cbar.ax.tick_params(labelsize=11)
    cbar.set_label('SHAP Value', fontsize=12, labelpad=5)

    # Configure axes
    ax.yaxis.set_major_locator(mticker.MaxNLocator(min_n_ticks=4, nbins=5))
    ax.xaxis.set_major_locator(mticker.MaxNLocator(min_n_ticks=3, nbins=5, integer=True))
    ax.set_xlabel(feature_name, fontsize=12, labelpad=4)
    ax.set_ylabel("Saturation Polarization (T)", fontsize=12, labelpad=4)
    ax.tick_params(axis='both', labelsize=11, pad=4)
    ax.grid(True, linestyle=':', alpha=0.35, linewidth=0.45)

# Save high-resolution figure
plt.savefig('Oliynyk_Grid-SHAP_Js_ETR.png',
            dpi=600,
            bbox_inches='tight',
            facecolor='white')
plt.show()

# Download in Colab environment
try:
    files.download('Oliynyk_Grid-SHAP_Js_ETR.png')
except:
    pass

# Gradient Boost Regressor

In [ ]:
# Train Gradient Boosting model with pre-optimized hyperparameters
best_model = GradientBoostingRegressor(
    n_estimators=180,
    learning_rate=0.04,
    max_depth=3,
    min_samples_split=6,
    min_samples_leaf=3,
    subsample=0.64,
    loss='squared_error',
    random_state=42
)
best_model.fit(X_train, y_train)

In [ ]:
if SAVE_MODEL:

# Save model to .pkl file
    joblib.dump(best_model, '3_oliynyk_SP_gbr.pkl')

    print("✅ Model saved as '3_oliynyk_SP_gbr.pkl'")
    try:
        files.download('3_oliynyk_SP_gbr.pkl')
    except:
        pass

In [ ]:
# Predict on test set
y_pred = best_model.predict(X_test)

# Calculate evaluation metrics
test_r2 = r2_score(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)

# Display metrics in formatted table
print("\nTest Set Evaluation Metrics:")
print("----------------------------------------")
print(f"| {'Metric':<10} | {'Value':>15} |")
print("|------------|----------------|")
print(f"| R²         | {test_r2:>15.4f} |")
print(f"| MAE        | {test_mae:>15.4f} |")
print(f"| MSE        | {test_mse:>15.4f} |")
print(f"| RMSE       | {test_rmse:>15.4f} |")
print("----------------------------------------")

In [ ]:
def evaluate_and_store(model, model_name, X_test, y_test, results_df):
    """Evaluate model and store metrics in results DataFrame."""

    # Predict on test set
    y_pred = model.predict(X_test)

    # Calculate regression metrics
    metrics = {
        'Model': model_name,
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R²': r2_score(y_test, y_pred)
    }

    # Append metrics to results DataFrame
    results_df = pd.concat([results_df, pd.DataFrame([metrics])], ignore_index=True)
    return results_df


# Evaluate GBR model and store results
model_results = evaluate_and_store(best_model, 'GBR', X_test, y_test, model_results)

# XGBoost Regressor

In [ ]:
# Train XGBoost model with pre-optimized hyperparameters
mean_target = float(y_train.mean())
best_model = xgb.XGBRegressor(
    n_estimators=350,
    max_depth=3,
    learning_rate=0.15,
    subsample=0.74,
    colsample_bytree=0.88,
    min_child_weight=1,
    gamma=0.01,
    alpha=0.01,
    base_score=mean_target,
    random_state=42
)
best_model.fit(X_train, y_train)

In [ ]:
if SAVE_MODEL:

# Save model to .pkl file
    joblib.dump(best_model, '4_oliynyk_SP_xgb.pkl')

    print("✅ Model saved as '4_oliynyk_SP_xgb.pkl'")
    try:
        files.download('4_oliynyk_SP_xgb.pkl')
    except:
        pass

In [ ]:
# Predict on test set
y_pred = best_model.predict(X_test)

# Calculate evaluation metrics
test_r2 = r2_score(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_mse = mean_squared_error(y_test, y_pred)
test_rmse = np.sqrt(test_mse)

# Display metrics in formatted table
print("\nTest Set Evaluation Metrics:")
print("----------------------------------------")
print(f"| {'Metric':<10} | {'Value':>15} |")
print("|------------|----------------|")
print(f"| R²         | {test_r2:>15.4f} |")
print(f"| MAE        | {test_mae:>15.4f} |")
print(f"| MSE        | {test_mse:>15.4f} |")
print(f"| RMSE       | {test_rmse:>15.4f} |")
print("----------------------------------------")

In [ ]:
if USE_EXTERNAL_VALIDATION:
    # Make predictions on external validation set (XGB)
    y_pred_xgb = best_model.predict(X_predict)

    # Save XGB predictions to results DataFrame
    results.loc[results["Model"] == "XGB", column_name] = y_pred_xgb
    print(f"XGB predictions saved: {y_pred_xgb}")
    print(results)
    print("\n")

    # Add experimental ground truth values
    y_true = [0.937, 0.884, 0.607, 0.794, 1.574]  # ← UPDATE WITH ACTUAL VALUES
    results["y_true"] = y_true * 2  # Duplicate for both models

    # Calculate errors
    results["error"] = results[column_name] - results["y_true"]
    results["abs_error"] = results["error"].abs()
    results["abs_percent_error_%"] = 100 * results["abs_error"] / results["y_true"]

    # Calculate average metrics per model
    for model in ["ETR", "XGB"]:
        mask = results["Model"] == model
        mae = results.loc[mask, "abs_error"].mean()
        mape = results.loc[mask, "abs_percent_error_%"].mean()
        print(f"{model}: MAE = {mae:.4f}, MAPE = {mape:.2f}%")

    # Save results to Excel
    results.to_excel(output_file, index=False, engine='openpyxl')
    print(f"✓ Results saved to {output_file}")

    # Download file (Colab only)
    try:
        files.download(output_file)
    except:
        pass

In [ ]:
def evaluate_and_store(model, model_name, X_test, y_test, results_df):
    """
    Evaluate model and store results in the results DataFrame
    Returns updated DataFrame
    """
    y_pred = model.predict(X_test)

    metrics = {
        'Model': model_name,
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R²': r2_score(y_test, y_pred)
    }

    results_df = pd.concat([results_df, pd.DataFrame([metrics])], ignore_index=True)
    return results_df

model_results = evaluate_and_store(best_model, 'XGB', X_test, y_test, model_results)

In [ ]:
# ====================================================================
# SHAP BEESWARM VISUALIZATION (XGBOOST)
# ====================================================================

# Create DMatrix for XGBoost native SHAP calculation
dmatrix_train = xgb.DMatrix(X_train, feature_names=X_train.columns.tolist())

# Calculate SHAP values using XGBoost native API
shap_values_raw = best_model.get_booster().predict(dmatrix_train, pred_contribs=True)
shap_values_matrix = shap_values_raw[:, :-1]  # Feature contributions
expected_value = float(shap_values_raw[0, -1])  # Base value

print(f"Shape SHAP values: {shap_values_matrix.shape}")
print(f"Expected value: {expected_value}")

# Create SHAP Explanation object
explanation = shap.Explanation(
    values=shap_values_matrix,
    base_values=np.full(len(X_train), expected_value),
    data=X_train.values,
    feature_names=X_train.columns.tolist()
)

# Set global font parameters for publication-quality plots
mpl.rcParams.update({
    'font.size': 23,
    'axes.labelsize': 23,
    'xtick.labelsize': 23,
    'ytick.labelsize': 23,
})

# Create figure and axis
fig, ax = plt.subplots(figsize=(8, 6))

# Generate beeswarm plot showing feature importance and value distribution
shap.plots.beeswarm(
    explanation,
    max_display=5,                     # Show top 5 most important features
    group_remaining_features=False,    # Don't group remaining features
    show=False,                        # Don't display yet (for customization)
    s=150,                             # Marker size
    ax=ax,
    plot_size=None
)

# Customize axis labels and ticks
ax.set_xlabel("SHAP Values", fontsize=23)
ax.tick_params(axis='x', labelsize=23, pad=2)
ax.tick_params(axis='y', which='both', length=0, pad=1, labelsize=23)

# Set custom x-axis tick positions
# desired_ticks = [-0.6, -0.3, 0, 0.3]
# ax.set_xticks(desired_ticks)

# Shift y-axis labels to the right for better readability
dx_pt = -3
offset = mtransforms.ScaledTranslation(dx_pt/72.0, 0, fig.dpi_scale_trans)
for t in ax.get_yticklabels():
    t.set_ha('right')
    t.set_transform(t.get_transform() + offset)

# Position y-axis spine
ax.spines['left'].set_position(('axes', 0.0))
ax.yaxis.set_ticks_position('left')

# Customize colorbar (feature value scale)
cax = fig.axes[-1]
cax.tick_params(pad=2, labelsize=23)
cax.set_ylabel("Feature Value", fontsize=23, labelpad=4)
cax.yaxis.set_label_coords(2.5, 0.5)

# Apply tight layout and save
fig.tight_layout(pad=0.2)
fig.savefig('Oliynyk_SHAP_beeswarm_Js_XGB.png', dpi=600, bbox_inches='tight', transparent=False)

plt.show()

# Download plot in Colab environment
try:
    files.download('Oliynyk_SHAP_beeswarm_Js_XGB.png')
except:
    pass  # Skip download for local execution

In [ ]:
# ====================================================================
# CUSTOM COLORMAP FUNCTIONS
# ====================================================================

def shifted_coolwarm_cmap(p, n=256):
    """Shift coolwarm colormap so position p becomes neutral center."""
    base = plt.get_cmap("coolwarm")
    xs = np.linspace(0, 1, n)

    cols = []
    for x in xs:
        if x <= p:
            t = 0.5 * (x / p) if p > 0 else 0.0
        else:
            t = 0.5 + 0.5 * ((x - p) / (1 - p)) if p < 1 else 1.0
        cols.append(base(t))

    return mcolors.ListedColormap(cols, name="coolwarm_shifted")


def make_coolwarm_saturated_with_center(values, vcenter=0.0, nbins=6, steps=(1, 2, 2.5, 3, 5, 10)):
    """Create norm, cmap, and ticks for SHAP with saturated colors and zero at neutral."""
    x = np.asarray(values, dtype=float)
    x = x[np.isfinite(x)]

    # Fallback for empty data
    if x.size == 0:
        norm = mcolors.Normalize(vmin=-1, vmax=1, clip=True)
        cmap = plt.get_cmap("coolwarm")
        ticks = np.array([-1, 0, 1], dtype=float)
        return norm, cmap, ticks

    data_min, data_max = float(np.min(x)), float(np.max(x))

    # Normalize by actual data range for saturated colors
    norm = mcolors.Normalize(vmin=data_min, vmax=data_max, clip=True)

    # Generate ticks with scientific steps
    loc = mticker.MaxNLocator(nbins=nbins, steps=list(steps))
    ticks = loc.tick_values(data_min, data_max)
    ticks = ticks[(ticks >= norm.vmin) & (ticks <= norm.vmax)]

    # Shift colormap to place vcenter at neutral
    if data_min < vcenter < data_max:
        p = (vcenter - data_min) / (data_max - data_min)
        cmap = shifted_coolwarm_cmap(p, n=256)

        # Ensure vcenter is in ticks
        if not np.any(np.isclose(ticks, vcenter)):
            ticks = np.sort(np.append(ticks, vcenter))
    else:
        cmap = plt.get_cmap("coolwarm")

    return norm, cmap, ticks

In [ ]:
# ====================================================================
# GRID-SHAP VISUALIZATION FOR XGBOOST
# ====================================================================

# Create DMatrix for XGBoost native SHAP calculation
dmatrix_train = xgb.DMatrix(X_train, feature_names=X_train.columns.tolist())

# Calculate SHAP values using XGBoost native API
shap_values_raw = best_model.get_booster().predict(dmatrix_train, pred_contribs=True)
shap_values_matrix = shap_values_raw[:, :-1]  # Feature contributions
expected_value = float(shap_values_raw[0, -1])  # Base value

# Create SHAP Explanation object
explanation = shap.Explanation(
    values=shap_values_matrix,
    base_values=np.full(len(X_train), expected_value),
    data=X_train.values,
    feature_names=X_train.columns.tolist()
)

print(f"Shape SHAP values: {shap_values_matrix.shape}")
print(f"Expected value: {expected_value}")

# Identify top 5 most important features by mean absolute SHAP value
mean_abs_shap_values = np.abs(explanation.values).mean(axis=0)
top5_indices = np.argsort(mean_abs_shap_values)[-5:][::-1]

# Create figure with 3x2 grid layout
plt.style.use('default')
fig = plt.figure(figsize=(8, 10))
gs = GridSpec(3, 2, figure=fig, wspace=0.7, hspace=0.5)

# Generate scatter plots for each top feature
for i, feature_index in enumerate(top5_indices):
    feature_name = X_train.columns[feature_index]
    row = i // 2
    col = i % 2

    ax = fig.add_subplot(gs[row, col])
    ax.set_facecolor('#f0f0f0')

    # Extract SHAP values for current feature
    shap_col = explanation[:, feature_index].values

    # Get normalized colormap with zero at neutral center
    norm, cmap_local, ticks = make_coolwarm_saturated_with_center(
        shap_col,
        vcenter=0.0,
        nbins=6
    )

    # Create scatter plot: feature value vs target, colored by SHAP
    sc = ax.scatter(
        X_train[feature_name],
        y_train,
        c=shap_col,
        cmap=cmap_local,
        norm=norm,
        alpha=0.9,
        s=40,
        edgecolor='black',
        linewidth=0.25,
        marker='o'
    )

    # Add colorbar with scientific notation
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    cbar = fig.colorbar(sc, cax=cax)

    # Configure colorbar ticks
    cbar.set_ticks(ticks)
    cbar.formatter = mticker.ScalarFormatter(useMathText=False)
    cbar.formatter.set_powerlimits((-100, 100))
    cbar.update_ticks()
    cbar.ax.tick_params(labelsize=11)
    cbar.set_label('SHAP Value', fontsize=12, labelpad=5)

    # Configure axes
    ax.yaxis.set_major_locator(mticker.MaxNLocator(min_n_ticks=4, nbins=5))
    ax.xaxis.set_major_locator(mticker.MaxNLocator(min_n_ticks=3, nbins=5, integer=True))
    ax.set_xlabel(feature_name, fontsize=12, labelpad=4)
    ax.set_ylabel("Saturation Polarization (T)", fontsize=12, labelpad=4)
    ax.tick_params(axis='both', labelsize=11, pad=4)
    ax.grid(True, linestyle=':', alpha=0.35, linewidth=0.45)

# Save high-resolution figure
plt.savefig('Oliynyk_Grid-SHAP_Js_XGB.png',
            dpi=600,
            bbox_inches='tight',
            facecolor='white')
plt.show()

# Download in Colab environment
try:
    files.download('Oliynyk_Grid-SHAP_Js_XGB.png')
except:
    pass  # Skip download for local execution

# **Results**

In [ ]:
# After evaluating all models, add this to display and save results:
def display_model_results(results_df):
    """Display and save model comparison results"""
    # Format display
    pd.options.display.float_format = '{:.4f}'.format
    results_df.set_index('Model', inplace=True)

    print("\nFinal Model Comparison:")
    print("="*60)
    print(results_df)
    print("="*60)

    # Highlight best scores
    def highlight_metrics(s):
        metrics = {'MAE': 'min', 'MSE': 'min', 'RMSE': 'min', 'R²': 'max'}
        styles = []
        for v in s:
            if s.name in metrics:
                if metrics[s.name] == 'min' and v == s.min():
                    styles.append('background-color: yellow')
                elif metrics[s.name] == 'max' and v == s.max():
                    styles.append('background-color: lightgreen')
                else:
                    styles.append('')
            else:
                styles.append('')
        return styles

    styled_df = results_df.style.apply(highlight_metrics)
    display(styled_df)

    # Save to Excel
    results_df.to_excel('Oliynyk_Js_metrics.xlsx')
    print("\nResults saved to 'Oliynyk_Js_metrics.xlsx'")


# Call this after evaluating all models
display_model_results(model_results)

# Download file (Colab only)
try:
    files.download('Oliynyk_Js_metrics.xlsx')
except:
    pass